This notebook show an example on how to run predictions using a pre-trained a lgsegmentor.

This example it uses the validation data 

In [1]:
import numpy as np
import tifffile
import napari
import leopardgecko.segmentor as lgs
import leopardgecko.utils as lgu

import logging
logging.basicConfig(level=logging.INFO)

Matplotlib is building the font cache; this may take a moment.


# Load data  for prediction and respective labels

In [2]:
data_to_pred = tifffile.imread("test_data/TS_0005_crop_val.tif")
labels_to_compare = tifffile.imread("test_data/TS_0005_ribos_membr_crop_val.tif")

In [3]:
print(data_to_pred.shape)

(256, 256, 256)


# Setup lg segmentor

In [4]:
#Create the class
lgsegmentor0 = lgs.cMultiAxisRotationsSegmentor.create_from_model("2023-09-29_ribos_membr.lgsegmod")

Check default settings and modify as needed

In [5]:
lgsegmentor0.NN1_pred_settings

namespace(quality='high',
          output_probs=True,
          clip_data=True,
          st_dev_factor=2.575,
          data_hdf5_path='/data',
          cuda_device=0,
          downsample=False,
          one_hot=False,
          prediction_axis='Z')

In [6]:
#Calculates consistency score and volume segmantics. Output filename must be h5 file
lgsegmentor0.NN1_consistencyscore_outpath="./consistency_score.h5" # delete file after
lgsegmentor0.NN1_volsegm_pred_path="./VS_prediction.h5"


Set CUDA device.

To change cuda device don't change with .cuda_device, but rather use setcudadevice(devnumber)

You may want to check usage and memory-usage with command `!nvidia-smi` to see what is available before starting training

In [7]:
!nvidia-smi

Fri Oct 20 11:23:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|


|   0  Tesla V100-PCIE...  Off  | 00000000:00:07.0 Off |                    0 |
| N/A   38C    P0    23W / 250W |      4MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  Off  | 00000000:00:08.0 Off |                    0 |
| N/A   31C    P0    24W / 250W |      4MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
|        ID   ID                        

In [8]:
lgsegmentor0.set_cuda_device(0)

Run the prediction and collect result

In [9]:
pred_labels = lgsegmentor0.predict(data_to_pred)

INFO:root:Setting up NN1 prediction
INFO:root:number of data sets to predict: 1
INFO:root:Data to predict index:0
INFO:root:Calculating mean of data...
INFO:root:Mean value: 3.376452088356018
INFO:root:Calculating standard deviation.
INFO:root:Std dev: 22.28497505498702. Calculating stats.
INFO:root:Lower bound: -54.00735867823556, upper bound: 60.7602628549476
INFO:root:Number of voxels above upper bound to be clipped 95283 - percentage 0.568%
INFO:root:Number of voxels below lower bound to be clipped 175185 - percentage 1.044%
INFO:root:Rescaling intensities.
INFO:root:Data is already in integer dtype, converting to float for rescaling.
INFO:root:Converting to uint8.
INFO:root:Loading model dictionary from file.
Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /home/ypu66991/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

INFO:root:Sending the U-Net model to device 0
INFO:root:Loading in the saved weights.
INFO:root:Volume to be rotated by 0 degrees
INFO:root:Predicting YX slices:
INFO:root:Free GPU memory is 31.66 GB. Batch size will be 4.
INFO:root:Predicting segmentation for volume of shape (256, 256, 256).
Prediction batch: 100%|██████████████████████████████| 64/64 [00:51<00:00,  1.23it/s]
INFO:root:Completed prediction. Now manipulating result before returning.
INFO:root:self.NN1_volsegm_pred_path provided:./VS_prediction.h5. Will merge and save to predicted labels using volumesegmantics method.
INFO:root:Saving data of shape (256, 256, 256, 3) to /tmp/tmpdcngjhzx/pred_0_YX_0.h5 with kwargs {}.
INFO:root:Saving data of shape (256, 256, 256) to /tmp/tmpdcngjhzx/pred_0_YX_labels_0.h5 with kwargs {}.
INFO:root:Predicting ZX slices:
INFO:root:Free GPU memory is 31.66 GB. Batch size will be 4.
INFO:root:Predicting segmentation for volume of shape (256, 256, 256).
Prediction batch: 100%|████████████████

[########################################] | 100% Completed | 31.12 ss


INFO:root:NN2 prediction complete.


# Visualise result

In [10]:
import leopardgecko.utils as lgu
data_cscore = lgu.read_h5_to_np("./consistency_score.h5")
pred_vs = lgu.read_h5_to_np("./VS_prediction.h5")

In [11]:
import napari
NV=napari.Viewer()
NV.add_image(data_to_pred)
NV.add_labels(labels_to_compare)
NV.add_labels(pred_labels, color={1:'green'})
NV.add_labels(pred_vs, color={1:'blue'})
NV.add_image(data_cscore)

<Image layer 'data_cscore' at 0x7f46843f4430>

# Compare against ground truth

Use leopardgecko metrics to get voxel-F1 (dice score)

In [12]:
import leopardgecko.metrics as lgm

In [13]:
lgm.MetricScoreOfVols_Dice(pred_labels,labels_to_compare)

INFO:root:MetricScoreOfVols_Dice()
INFO:root:Number of class segmentations in first volume 3
INFO:root:Number of class segmentations in second volume 3
INFO:root:Calculating using numpy
INFO:root:dicescores = {1: 0.5188051691479704, 2: 0.5507384751326172}


(0.5347718221402937, {1: 0.5188051691479704, 2: 0.5507384751326172})

# Run the prediction in blocks?

Note that the way that this works is by using the block map function, so the CS output (to a file) will not be assembled appropriately like the resulting data is.

In [14]:
lgsegmentor0.NN1_consistencyscore_outpath=None